In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#changes to the pokemon list in order to use their urls

#replace nidoran symbols with -f, -m
#remove farfetch'd apostrophe (and sirfetch'd)
#replace mr.mime with mr-mime, mime jr. with mime-jr, same with mr. rime
#flabébé with flabebe
#type: null with type-null

#summary: remove apostrophe, colons, periods
#replace spaces with -, é with e
#nidoran symbols done manually
pokemon_list = []
f = open('pokemon list.txt')
for x in f:
    line = x.replace("Ã©", "e").replace(" ","-").replace("'","").replace(":","").replace(".","")
    pokemon_list.append(line)

df = pd.DataFrame()
for pokemon in pokemon_list:
    pokemon = pokemon.strip()
    url = 'https://pokemondb.net/pokedex/' + pokemon
    page = requests.get(url)
    #just to see which pokemon doesnt work
    print(pokemon)

    soup = BeautifulSoup(page.text,features="html.parser")

    #since each pokemon has a different number of forms, the table we are looking for later will not always be the second one,
    #thus we need to count each pokemon's number of forms
    forms_table = soup.find_all('div', class_ = 'sv-tabs-tab-list')[0]

    forms = forms_table.find_all('a')
    forms_count = len(forms)

    table = soup.find_all('div', class_ = 'grid-col span-md-12 span-lg-8')[forms_count]

    for br in table.find_all("br"):
        br.replace_with("\n")

    games = table.find_all('th')
    games2 = [title.text.splitlines() for title in games]
    games3 = [item for sublist in games2 for item in sublist]
    games3.insert(0,'Name')

    locations = table.find_all('tr')

    #have a dictionary, with keys being the games, values being their location
    #dictionary should have 38 elements 
    values = [None] * 38
    dictionary = dict(zip(games3, values)) 

    #go through locations, for each game, put the data in the dictionary

    table_rows = [title.text.strip() for title in locations]

    pairs_list = []

    for x in table_rows:
        newlines = len(x.splitlines()) - 1

        for y in range(newlines):
            pair = {x.splitlines()[y]:x.splitlines()[newlines]}
            pairs_list.append(pair)

    #add values to dictionary
    for x in pairs_list:
        dictionary.update(x)

    pair = {'Name':pokemon}
    dictionary.update(pair)

    #this table will find us each pokemon's type(s)
    table2 = soup.find_all('table', class_ = 'vitals-table')[0]

    type1 = table2.find_all('a')[0].text
    type2 = table2.find_all('a')[1].text

    types_list = ['Normal','Fire','Water','Electric','Grass','Ice','Fighting','Poison','Ground','Flying','Psychic','Bug','Rock','Ghost','Dragon','Dark','Steel','Fairy']

    #if the pokemon only has one type, type2 will be something else, we can just set it to 'None'
    if type2 not in types_list:
        type2 = 'None'

    pair1 = {'Type1': type1}
    pair2 = {'Type2': type2}
    dictionary.update(pair1)
    dictionary.update(pair2)

    temp_df = pd.DataFrame([dictionary])

    df = pd.concat([df, temp_df], ignore_index=True)


#rearrange the dataframe to make Name, Type1, Type2 be the first 3 columns respectively
names = df.pop("Name")
df.insert(0,"Name", names)
type1s = df.pop("Type1")
df.insert(1,"Type1", type1s)
type2s = df.pop("Type2")
df.insert(2,"Type2", type2s)

df

Bulbasaur
Ivysaur
Venusaur
Charmander
Charmeleon
Charizard
Squirtle
Wartortle
Blastoise
Caterpie
Metapod
Butterfree
Weedle
Kakuna
Beedrill
Pidgey
Pidgeotto
Pidgeot
Rattata
Raticate
Spearow
Fearow
Ekans
Arbok
Pikachu
Raichu
Sandshrew
Sandslash
Nidoran-F
Nidorina
Nidoqueen
Nidoran-M
Nidorino
Nidoking
Clefairy
Clefable
Vulpix
Ninetales
Jigglypuff
Wigglytuff
Zubat
Golbat
Oddish
Gloom
Vileplume
Paras
Parasect
Venonat
Venomoth
Diglett
Dugtrio
Meowth
Persian
Psyduck
Golduck
Mankey
Primeape
Growlithe
Arcanine
Poliwag
Poliwhirl
Poliwrath
Abra
Kadabra
Alakazam
Machop
Machoke
Machamp
Bellsprout
Weepinbell
Victreebel
Tentacool
Tentacruel
Geodude
Graveler
Golem
Ponyta
Rapidash
Slowpoke
Slowbro
Magnemite
Magneton
Farfetchd
Doduo
Dodrio
Seel
Dewgong
Grimer
Muk
Shellder
Cloyster
Gastly
Haunter
Gengar
Onix
Drowzee
Hypno
Krabby
Kingler
Voltorb
Electrode
Exeggcute
Exeggutor
Cubone
Marowak
Hitmonlee
Hitmonchan
Lickitung
Koffing
Weezing
Rhyhorn
Rhydon
Chansey
Tangela
Kangaskhan
Horsea
Seadra
Goldeen
Seakin

,Name,Type1,Type2,Red,Blue,Yellow,Gold,Silver,Crystal,Ruby,...,Ultra Moon,Let's Go Pikachu,Let's Go Eevee,Sword,Shield,Brilliant Diamond,Shining Pearl,Legends: Arceus,Scarlet,Violet
0,Bulbasaur,Grass,Poison,Pallet Town,Pallet Town,Cerulean City,Trade/migrate from another game,Trade/migrate from another game,Trade/migrate from another game,Trade/migrate from another game,...,Route 2,"Cerulean City, Viridian Forest","Cerulean City, Viridian Forest",Trade/migrate from another game,Trade/migrate from another game,"Grassland Cave, Swampy Cave, Riverbank Cave, S...","Grassland Cave, Swampy Cave, Riverbank Cave, S...",Not available in this game,Not available in this game,Not available in this game
1,Ivysaur,Grass,Poison,Evolve Bulbasaur,Evolve Bulbasaur,Evolve Bulbasaur,Trade/migrate from another game,Trade/migrate from another game,Trade/migrate from another game,Trade/migrate from another game,...,Evolve Bulbasaur,Evolve Bulbasaur,Evolve Bulbasaur,Trade/migrate from another game,Trade/migrate from another game,Evolve Bulbasaur,Evolve Bulbasaur,Not available in this game,Not available in this game,Not available in this game
2,Venusaur,Grass,Poison,Evolve Bulbasaur/Ivysaur,Evolve Bulbasaur/Ivysaur,Evolve Bulbasaur/Ivysaur,Trade/migrate from another game,Trade/migrate from another game,Trade/migrate from another game,Trade/migrate from another game,...,Evolve Bulbasaur/Ivysaur,Evolve Bulbasaur/Ivysaur,Evolve Bulbasaur/Ivysaur,Trade/migrate from another game,Trade/migrate from another game,Evolve Bulbasaur/Ivysaur,Evolve Bulbasaur/Ivysaur,Not available in this game,Not available in this game,Not available in this game
3,Charmander,Fire,None,Pallet Town,Pallet Town,Route 24,Trade/migrate from another game,Trade/migrate from another game,Trade/migrate from another game,Trade/migrate from another game,...,Route 3,"Route 3, 4, 24, Rock Tunnel","Route 3, 4, 24, Rock Tunnel",Trade/migrate from another game,Trade/migrate from another game,"Volcanic Cave, Sandsear Cave, Typhlo Cavern","Volcanic Cave, Sandsear Cave, Typhlo Cavern",Not available in this game,Trade/migrate from another game,Trade/migrate from another game
4,Charmeleon,Fire,None,Evolve Charmander,Evolve Charmander,Evolve Charmander,Trade/migrate from another game,Trade/migrate from another game,Trade/migrate from another game,Trade/migrate from another game,...,Evolve Charmander,Evolve Charmander,Evolve Charmander,Trade/migrate from another game,Trade/migrate from another game,Evolve Charmander,Evolve Charmander,Not available in this game,Trade/migrate from another game,Trade/migrate from another game
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,Iron-Valiant,Fairy,Fighting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Trade/migrate from another game,The Great Crater of Paldea
1006,Koraidon,Fighting,Dragon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Poco Path, The Great Crater of Paldea",Trade/migrate from another game
1007,Miraidon,Electric,Dragon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Trade/migrate from another game,"Poco Path, The Great Crater of Paldea"
1008,Walking-Wake,Water,Dragon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Trade/migrate from another game,Trade/migrate from another game
